In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states

# Dependencies
import requests
import json

# Census API Key
from config import api_key,gkey
c = Census(api_key, year=2018)

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

In [3]:
 mdcheck = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"),
           geo={'for': 'zip code tabulation area:*',
                        'in': 'state:{}'.format(states.IN.fips)})

 # Convert to DataFrame
census_pd2 = pd.DataFrame(mdcheck)

census_pd2.head()
 len(census_pd2)

775

In [4]:
# Add in Poverty Rate (Poverty Count / Population)
census_pd2["Poverty Rate"] = 100 * \
    census_pd2["B17001_002E"].astype(
        int) / census_pd2["B01003_001E"].astype(int)

In [5]:
census_pd2.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,state,zip code tabulation area,Poverty Rate
0,ZCTA5 46511,52570.0,4254.0,46.9,32929.0,348.0,18,46511,8.180536
1,ZCTA5 46526,54187.0,32513.0,36.0,24312.0,4495.0,18,46526,13.825239
2,ZCTA5 46528,63537.0,27165.0,32.6,26499.0,2439.0,18,46528,8.978465
3,ZCTA5 46544,47778.0,31359.0,38.0,25268.0,4324.0,18,46544,13.788705
4,ZCTA5 46553,62829.0,3212.0,36.7,24039.0,78.0,18,46553,2.428394


In [6]:
# Column Reordering
census_pd2 = census_pd2.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})
# Final DataFrame
census_pd_final = census_pd2[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

# Visualize
print(len(census_pd_final))
census_pd_final.head()

775


,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,46511,4254.0,46.9,52570.0,32929.0,348.0,8.180536
1,46526,32513.0,36.0,54187.0,24312.0,4495.0,13.825239
2,46528,27165.0,32.6,63537.0,26499.0,2439.0,8.978465
3,46544,31359.0,38.0,47778.0,25268.0,4324.0,13.788705
4,46553,3212.0,36.7,62829.0,24039.0,78.0,2.428394


In [7]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd_final.to_csv("census_data_indiana_2014.csv", encoding="utf-8", index=False)

In [8]:
# Read CSV file into DataFrame df
uni_df_raw = pd.read_csv('universities.csv', index_col=0)

# Show dataframe
uni_df_raw.reset_index().head()

,School,City,State
0,ANCILLA COLLEGE,DONALDSON,IN
1,ANDERSON UNIVERSITY,ANDERSON,IN
2,BALL STATE UNIVERSITY,MUNCIE,IN
3,BETHEL COLLEGE,MISHAWAKA,IN
4,BUTLER UNIVERSITY,INDIANAPOLIS,IN


In [9]:
# Read CSV file into DataFrame df
df = pd.read_csv('indiana_k12.csv', index_col=0)

# Show dataframe
city_list = df[["CITY"]]
city_list_dedup = city_list.drop_duplicates()
city_list_dedup

,CITY
0,LOGANSPORT
1,FOUNTAIN CITY
2,NOBLESVILLE
3,BROWNSBURG
5,ELKHART
...,...
1833,OTWELL
1838,SHELBURN
1846,LEAVENWORTH
1847,NEW ROSS


In [10]:
city_list_dedup['Lat']=" "
city_list_dedup['Lng']=" "

city_list_dedup

<ipython-input-10-61a864764472>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_list_dedup['Lat']=" "
<ipython-input-10-61a864764472>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_list_dedup['Lng']=" "


,CITY,Lat,Lng
0,LOGANSPORT,,
1,FOUNTAIN CITY,,
2,NOBLESVILLE,,
3,BROWNSBURG,,
5,ELKHART,,
...,...,...,...
1833,OTWELL,,
1838,SHELBURN,,
1846,LEAVENWORTH,,
1847,NEW ROSS,,


In [11]:
# Run a request to endpoint and convert result to json

lonely_city = []


for index,row in city_list_dedup.iterrows():
    
    target_city_row = row["CITY"]
    
    target_city = f"{target_city_row}, Indiana"

# Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_city, gkey)
    
    geo_data = requests.get(target_url).json()
    
# Extract latitude and longitude
    try:
        city_list_dedup.loc[index,"Lat"] = geo_data["results"][0]["geometry"]["location"]["lat"]
        city_list_dedup.loc[index,"Lng"] = geo_data["results"][0]["geometry"]["location"]["lng"]
        print(f"Record found at {target_city}")
    except (KeyError, IndexError):
        print(f"Record could not be found at {target_city}")
        lonely_city = target_city_row

C:\Users\andre\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\andre\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Record found at LOGANSPORT, Indiana
Record found at FOUNTAIN CITY, Indiana
Record found at NOBLESVILLE, Indiana
Record found at BROWNSBURG, Indiana
Record found at ELKHART, Indiana
Record found at INDIANAPOLIS, Indiana
Record found at VALPARAISO, Indiana
Record found at AVON, Indiana
Record found at SELLERSBURG, Indiana
Record found at FISHERS, Indiana
Record found at LAKE STATION, Indiana
Record found at SCIPIO, Indiana
Record found at HAMMOND, Indiana
Record found at NASHVILLE, Indiana
Record found at HAYDEN, Indiana
Record found at ST LEON, Indiana
Record found at ZIONSVILLE, Indiana
Record found at CHURUBUSCO, Indiana
Record found at ANDERSON, Indiana
Record found at ROSSVILLE, Indiana
Record found at MARION, Indiana
Record found at GREENWOOD, Indiana
Record found at MARENGO, Indiana
Record found at WOLCOTT, Indiana
Record found at ARGOS, Indiana
Record found at VINCENNES, Indiana
Record found at CLAY CITY, Indiana
Record found at ANGOLA, Indiana
Record found at RICHMOND, Indiana
R

Record found at N MANCHESTER, Indiana
Record found at CANNELTON, Indiana
Record found at ELLETTSVILLE, Indiana
Record found at MOORESVILLE, Indiana
Record found at MONON, Indiana
Record found at DENVER, Indiana
Record found at SULLIVAN, Indiana
Record found at MILAN, Indiana
Record found at WINCHESTER, Indiana
Record found at OWENSVILLE, Indiana
Record found at CENTERVILLE, Indiana
Record found at MARSHALL, Indiana
Record found at FORTVILLE, Indiana
Record found at OOLITIC, Indiana
Record found at WOODBURN, Indiana
Record found at DYER, Indiana
Record found at WADESVILLE, Indiana
Record found at ROCKPORT, Indiana
Record found at HENRYVILLE, Indiana
Record found at SCOTTSBURG, Indiana
Record found at REDKEY, Indiana
Record found at WOLF LAKE, Indiana
Record found at HAMLET, Indiana
Record found at FRANCESVILLE, Indiana
Record found at WEST HARRISON, Indiana
Record found at BERNE, Indiana
Record found at WHITING, Indiana
Record found at BICKNELL, Indiana
Record found at WINAMAC, Indiana


In [12]:
city_list_dedup

,CITY,Lat,Lng
0,LOGANSPORT,40.754484,-86.356666
1,FOUNTAIN CITY,39.955603,-84.917462
2,NOBLESVILLE,40.045592,-86.008595
3,BROWNSBURG,39.843377,-86.397774
5,ELKHART,41.681208,-85.965625
...,...,...,...
1833,OTWELL,38.454773,-87.092228
1838,SHELBURN,39.178374,-87.39363
1846,LEAVENWORTH,38.199787,-86.344144
1847,NEW ROSS,39.964766,-86.71445


In [14]:
city_list_dedup2 = city_list_dedup.set_index('CITY')
city_list_dedup2
merged_df = uni_df_raw.join(city_list_dedup2, on='City')
merged_df2 = merged_df.reset_index()

# Note to avoid any issues later, use encoding="utf-8"
merged_df2.to_csv("city_lat_long_ind.csv", encoding="utf-8", index=False)

#If rerunning this, manual input for 
# ST MARY-OF-THE-WOODS	IN	39.51087	-87.46186
# NORTH MANCHESTER	IN	41.011112	-85.7625
# DONALDSON	IN	41.359764	-86.44433


In [20]:
# Read CSV file into DataFrame df -- need to read-in due to manual updates - can remove if fixedla
lat_long_df = pd.read_csv('city_lat_long_ind.csv', index_col=0)
lat_long_df

,City,State,Lat,Lng
School,,,,
ANCILLA COLLEGE,DONALDSON,IN,NaN,NaN
ANDERSON UNIVERSITY,ANDERSON,IN,40.105320,-85.680254
BALL STATE UNIVERSITY,MUNCIE,IN,40.193377,-85.386360
BETHEL COLLEGE,MISHAWAKA,IN,41.661993,-86.158616
BUTLER UNIVERSITY,INDIANAPOLIS,IN,39.768403,-86.158068
...,...,...,...,...
UNIVERSITY OF NOTRE DAME,SOUTH BEND,IN,41.676355,-86.251990
UNIVERSITY OF SOUTHERN INDIANA,EVANSVILLE,IN,37.971559,-87.571090
VALPARAISO UNIVERSITY,VALPARAISO,IN,41.473095,-87.061141


In [21]:
lat_long_df['place_id']=" "
lat_long_df

,City,State,Lat,Lng,place_id
School,,,,,
ANCILLA COLLEGE,DONALDSON,IN,NaN,NaN,
ANDERSON UNIVERSITY,ANDERSON,IN,40.105320,-85.680254,
BALL STATE UNIVERSITY,MUNCIE,IN,40.193377,-85.386360,
BETHEL COLLEGE,MISHAWAKA,IN,41.661993,-86.158616,
BUTLER UNIVERSITY,INDIANAPOLIS,IN,39.768403,-86.158068,
...,...,...,...,...,...
UNIVERSITY OF NOTRE DAME,SOUTH BEND,IN,41.676355,-86.251990,
UNIVERSITY OF SOUTHERN INDIANA,EVANSVILLE,IN,37.971559,-87.571090,
VALPARAISO UNIVERSITY,VALPARAISO,IN,41.473095,-87.061141,


In [22]:
lat_long_df2 =lat_long_df.reset_index() 
lat_long_df2

,School,City,State,Lat,Lng,place_id
0,ANCILLA COLLEGE,DONALDSON,IN,NaN,NaN,
1,ANDERSON UNIVERSITY,ANDERSON,IN,40.105320,-85.680254,
2,BALL STATE UNIVERSITY,MUNCIE,IN,40.193377,-85.386360,
3,BETHEL COLLEGE,MISHAWAKA,IN,41.661993,-86.158616,
4,BUTLER UNIVERSITY,INDIANAPOLIS,IN,39.768403,-86.158068,
...,...,...,...,...,...,...
58,UNIVERSITY OF NOTRE DAME,SOUTH BEND,IN,41.676355,-86.251990,
59,UNIVERSITY OF SOUTHERN INDIANA,EVANSVILLE,IN,37.971559,-87.571090,
60,VALPARAISO UNIVERSITY,VALPARAISO,IN,41.473095,-87.061141,
61,VINCENNES UNIVERSITY,VINCENNES,IN,38.677269,-87.528633,


In [23]:
for index,row in lat_long_df2.iterrows():
    
    target_lat = row["Lat"]
    target_lng = row["Lng"]
    target_school = row["School"]

    # geocoordinates
    target_coordinates = f"{target_lat}, {target_lng}"
    target_search = f"{target_school}"
    target_radius = 10000

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "key": gkey
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    places_data = response.json()
    
    # Extract latitude and longitude
    try:
        lat_long_df2.loc[index,"place_id"] = places_data["results"][0]["place_id"]
        print(f"Record found at {target_school}")
    except (KeyError, IndexError):
        print(f"Record could not be found at {target_school} at {target_lat} and {target_lng}")

Record could not be found at ANCILLA COLLEGE at nan and nan
Record found at ANDERSON UNIVERSITY
Record found at BALL STATE UNIVERSITY
Record found at BETHEL COLLEGE
Record found at BUTLER UNIVERSITY
Record found at CALUMET COLLEGE
Record found at CONCORDIA THEOLOGICAL SEMINARY
Record could not be found at DAVENPORT COLLEGE at 41.7533819 and -86.11083769999999
Record found at DEACONESS HOSPITAL SCHOOL OF NURSING
Record found at DEPAUW UNIVERSITY
Record found at EARLHAM COLLEGE
Record found at FAIRHAVEN COLLEGE
Record found at FRANKLIN COLLEGE
Record found at GOSHEN COLLEGE
Record found at GRACE COLLEGE
Record found at GRACE THEOLOGICAL SEMINARY
Record found at HANOVER COLLEGE
Record found at HOLY CROSS JUNIOR COLLEGE
Record found at HUNTINGTON COLLEGE
Record found at HYLES-ANDERSON COLLEGE
Record found at INDIANA INSTITUTE OF TECHNOLOGY
Record found at INDIANA STATE UNIVERSITY
Record found at INDIANA UNIVERSITY AT KOKOMO
Record found at INDIANA UNIVERSITY AT SOUTH BEND
Record found at I

In [19]:
# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 40.0369332,
                    "lng": -86.90667119999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 40.03832462989272,
                        "lng": -86.90528582010727
                    },
                    "southwest": {
                        "lat": 40.03562497010728,
                        "lng": -86.90798547989272
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/school-71.png",
            "icon_background_color": "#7B9EB0",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/school_pinlet",
            "name": "Wabash College",
            "opening_hours": {
                "open_now": false
      

In [24]:
lat_long_df2

,School,City,State,Lat,Lng,place_id,ZIP
0,ANCILLA COLLEGE,DONALDSON,IN,NaN,NaN,,NaN
1,ANDERSON UNIVERSITY,ANDERSON,IN,40.105320,-85.680254,,ChIJvRvrHQzZFIgRaG89vy-ipXg
2,BALL STATE UNIVERSITY,MUNCIE,IN,40.193377,-85.386360,,ChIJIR58aWY9FYgR9ImfGJvu4OQ
3,BETHEL COLLEGE,MISHAWAKA,IN,41.661993,-86.158616,,ChIJf2dC58HNFogRb1W460Xop8g
4,BUTLER UNIVERSITY,INDIANAPOLIS,IN,39.768403,-86.158068,,ChIJr8OliPpTa4gRPkUtyy7TxQM
...,...,...,...,...,...,...,...
58,UNIVERSITY OF NOTRE DAME,SOUTH BEND,IN,41.676355,-86.251990,,ChIJUe6QWPnLFogRinKUEbLx5oY
59,UNIVERSITY OF SOUTHERN INDIANA,EVANSVILLE,IN,37.971559,-87.571090,,ChIJc1S25M7ScYgRzQ1gMnNAqQI
60,VALPARAISO UNIVERSITY,VALPARAISO,IN,41.473095,-87.061141,,ChIJ2zyuvGqaEYgRGLo8A2GYSGY
61,VINCENNES UNIVERSITY,VINCENNES,IN,38.677269,-87.528633,,ChIJcWktmGz_bYgRRK9-5z_VLOg


In [41]:
for index,row in lat_long_df2.iterrows():
    
    target_place_id = row["ZIP"]
    

# Build the endpoint URL
    target_url = (f'https://maps.googleapis.com/maps/api/place/details/json?place_id={target_place_id}&key={gkey}')
    
    zip_data = requests.get(target_url).json()
    
# Extract latitude and longitude
    try:
        if zip_data["result"]["address_components"][7]["long_name"] == "United States":
            lat_long_df2.loc[index,"Zip Code"] = zip_data["result"]["address_components"][8]["long_name"]
        else:
            lat_long_df2.loc[index,"Zip Code"] = zip_data["result"]["address_components"][7]["long_name"]
        print(f"Record found at {target_place_id}")
    except (KeyError, IndexError):
#         if IndexError:
#             lat_long_df2.loc[index,"Zip Code"] = zip_data["result"]["address_components"][6]["long_name"]
#             print(f"Record found with 6 index for {target_place_id}")
#         else:
        print(f"Record could not be found for{target_place_id}")


Record could not be found fornan
Record found at ChIJvRvrHQzZFIgRaG89vy-ipXg
Record found at ChIJIR58aWY9FYgR9ImfGJvu4OQ
Record found at ChIJf2dC58HNFogRb1W460Xop8g
Record found at ChIJr8OliPpTa4gRPkUtyy7TxQM
Record found at ChIJ4-zpriPZEYgRVKD5Q2T6y2k
Record found at ChIJBZUXQdXiFYgR41Zsgf0DWw0
Record could not be found fornan
Record found at ChIJC31ykRrVcYgR82uJ6BZUfyA
Record found at ChIJp5coR7LfbIgRmbxAHZy330s
Record found at ChIJPbhwJrEBQIgRDMD-hfH0UW0
Record found at ChIJU8qfPU-jEYgRnX0LaK6qvc4
Record found at ChIJuWxvGOZua4gR79lD0GBEX7c
Record found at ChIJ0fC3UPTsFogRswXigO3sjZA
Record found at ChIJoQ-m_6GdFogRPu_aJZRUL_E
Record found at ChIJE4WWBMSdFogRcBmzi2VR5Hc
Record found at ChIJRyZwm94vaogRisLnrKcsabY
Record found at ChIJ275JTtHSFogRg_purimcnLU
Record found at ChIJk_5rlZbJFYgRbenfs0Si_Mc
Record found at ChIJXYPfxb3lEYgRcqOc71zBdHY
Record found at ChIJS0ChKLjkFYgR0t6rco0F5Oc
Record found at ChIJQ6w2DTplbYgReJ_3x_lCBPM
Record found at ChIJjcWkJ7OFFIgREhty2rlkKDo
Record fou

In [44]:
#

In [45]:
lat_long_df2

,School,City,State,Lat,Lng,place_id,ZIP,Zip Code
0,ANCILLA COLLEGE,DONALDSON,IN,NaN,NaN,,NaN,NaN
1,ANDERSON UNIVERSITY,ANDERSON,IN,40.105320,-85.680254,,ChIJvRvrHQzZFIgRaG89vy-ipXg,46012
2,BALL STATE UNIVERSITY,MUNCIE,IN,40.193377,-85.386360,,ChIJIR58aWY9FYgR9ImfGJvu4OQ,47306
3,BETHEL COLLEGE,MISHAWAKA,IN,41.661993,-86.158616,,ChIJf2dC58HNFogRb1W460Xop8g,46545
4,BUTLER UNIVERSITY,INDIANAPOLIS,IN,39.768403,-86.158068,,ChIJr8OliPpTa4gRPkUtyy7TxQM,46208
...,...,...,...,...,...,...,...,...
58,UNIVERSITY OF NOTRE DAME,SOUTH BEND,IN,41.676355,-86.251990,,ChIJUe6QWPnLFogRinKUEbLx5oY,NaN
59,UNIVERSITY OF SOUTHERN INDIANA,EVANSVILLE,IN,37.971559,-87.571090,,ChIJc1S25M7ScYgRzQ1gMnNAqQI,47712
60,VALPARAISO UNIVERSITY,VALPARAISO,IN,41.473095,-87.061141,,ChIJ2zyuvGqaEYgRGLo8A2GYSGY,46383
61,VINCENNES UNIVERSITY,VINCENNES,IN,38.677269,-87.528633,,ChIJcWktmGz_bYgRRK9-5z_VLOg,47591


,School,City,State,Lat,Lng,place_id,ZIP,Zip Code
27,INDIANA UNIVERSITY-BLOOMINGTON,BLOOMINGTON,IN,39.165325,-86.526386,,ChIJM62JZ8JmbIgRNtQVs4ugmTQ,7000
